In [2]:
from tensorflow import keras
from tensorflow.keras import layers

from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import LSTM
from keras.callbacks import ModelCheckpoint

import numpy as np
import random
import io
import json
import re

In [3]:
# path = keras.utils.get_file(
#     "nietzsche.txt", origin="https://s3.amazonaws.com/text-datasets/nietzsche.txt"
# )
path = './manele.json'


In [4]:
text = []
with io.open(path, encoding="utf-8") as f:
    file = json.load(f)
    for manea in file:
        for lyric in manea['lyrics']:
            text.append(lyric)
text = ''.join(text)


In [5]:
text = text.lower()
to_replace = list('!"$&()*+/:;<=>@[]^_~{}')
to_replace.append("refren")
to_replace.append("ref")
to_replace.append("x2")
to_replace.append("2x")
to_replace.append("florin salam")
to_replace.append("salam")
to_replace.append("bis")

for word in to_replace:
    text = text.replace(word, '')
    pass
text = re.sub(r'\n\s*\n', '\n\n', text)
text = re.sub(r'\.{4,}', '...', text)
print(text)
print("Corpus length:", len(text))

chars = sorted(list(set(text)))
print(chars)
print("Total chars:", len(chars))
char_indices = dict((c, i) for i, c in enumerate(chars))
indices_char = dict((i, c) for i, c in enumerate(chars))

# cut the text in semi-redundant sequences of maxlen characters
maxlen = 100
step = 3
sentences = []
next_chars = []
for i in range(0, len(text) - maxlen, step):
    sentences.append(text[i : i + maxlen])
    next_chars.append(text[i + maxlen])
print("Number of sequences:", len(sentences))
x = np.zeros((len(sentences), maxlen, len(chars)), dtype=np.bool)
y = np.zeros((len(sentences), len(chars)), dtype=np.bool)
for i, sentence in enumerate(sentences):
    for t, char in enumerate(sentence):
        x[i, t, char_indices[char]] = 1
    y[i, char_indices[next_chars[i]]] = 1


stau la geam plangand
poate astazi ai sa vii
tu stii k ti'as da tot ce am
sa te vad din nou zambind. 
am luat 7 trandafiri
si tin loc de amintiri despre tine
si ii pun pe perna ta si ma minte inima
doar pe tine... 
prin lume ratacesc cu stelele vorbesc
unde ejti oare? 
si nimeni nu va jtii ce mult noi ne-am iubit
si cat ma doare... 
prin lume ratacesc cu stelele vorbesc
unde ejti oare? 
si nimeni nu va jtii ce mult noi ne-am iubit
si cat ma doare... 
n'ai venit nici azi
si cat de mult te-am astept
in sufletel am doar necaz iar tu de mine ai uitat. 
am luat 7 trandafiri
si tin loc de amintiri despre tine.
7 inimi am in mine.. 
dumnezeu tine cu mine... 
unde umblu peste tot... 
dumnezeu imi da mare noroc... 
sufletul si inima din mine... 
eu le'mpart si ajut pe oricine... 
pre
inima mai buna ca a mea... 
te legi la rana cu ea... 
7 mari si 7 tari n'ai sa mai gasesti ca ea... 

7 inimi am in mine.. 
dumnezeu tine cu mine... 
unde umblu peste tot... 
dumnezeu imi da mare noroc... 
inima me

In [6]:
model = Sequential()
model.add(LSTM(256, input_shape=(maxlen, len(chars)), return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(256))
model.add(Dropout(0.2))
model.add(Dense(y.shape[1], activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam')

In [7]:
def sample(preds, temperature=1.0):
    # helper function to sample an index from a probability array
    preds = np.asarray(preds).astype("float64")
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)

In [8]:
# load the network weights
filename = "weights-10-0.9841.hdf5"
model.load_weights(filename)
model.compile(loss='categorical_crossentropy', optimizer='adam')

In [10]:
generated_length = 300
start_index = random.randint(0, len(text) - maxlen - 1)
for diversity in [0.2]:
    # print("...Diversity:", diversity)

    generated = ""
    original_sentence = text[start_index : start_index + maxlen]
    sentence = original_sentence
    # print('...Generating with seed:\n "' + sentence + '"')

    for i in range(generated_length):
        x_pred = np.zeros((1, maxlen, len(chars)))
        for t, char in enumerate(sentence):
            x_pred[0, t, char_indices[char]] = 1.0
        preds = model.predict(x_pred, verbose=0)[0]
        next_index = sample(preds, diversity)
        next_char = indices_char[next_index]
        sentence = sentence[1:] + next_char
        generated += next_char

    print(original_sentence + generated)

a-ma iubirea mea  

frumoasa stea intoarce-te la mine 
chiar daca stiu ca plange cerul pentru tine
la mine ce ma intrebe

pentru ca am aveti numai pe mine

sa fii cand viata pe altcineva

am sa plang pentru tine
sa fii asa cum faci de mult

daca nu mai vrei sa prea sa te mai pot avea
chiar daca as putea sa mai dau de tine
sa te intorci din nou
stii ca nu sunt de perea ta

sunt si paca tu ma vada t

